In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Hatua ya 1: Fafanua Miundo ya Pydantic kwa Matokeo Yaliyopangiliwa

Miundo hii inafafanua **schema** ambayo mawakala watarudisha. Kutumia `response_format` na Pydantic huhakikisha:
- ✅ Utoaji wa data salama kwa aina
- ✅ Uthibitishaji wa moja kwa moja
- ✅ Hakuna makosa ya uchambuzi kutoka kwa majibu ya maandishi ya kawaida
- ✅ Urahisi wa kuelekeza masharti kulingana na sehemu


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Hatua ya 2: Unda Zana ya Kuhifadhi Hoteli

Zana hii ndiyo **availability_agent** itakayotumia kuangalia kama vyumba vinapatikana. Tunatumia kivinjari `@ai_function` ili:
- Kubadilisha kazi ya Python kuwa zana inayoweza kuitwa na AI
- Kuunda kiotomatiki JSON schema kwa LLM
- Kushughulikia uthibitishaji wa vigezo
- Kuwezesha mawakala kuitumia kiotomatiki

Kwa mfano huu:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Vyumba vinapatikana ✅
- **Miji mingine yote** → Hakuna vyumba ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Hatua ya 3: Fafanua Kazi za Masharti kwa Usambazaji

Kazi hizi zinachunguza jibu la wakala na kuamua njia ya kufuata katika mtiririko wa kazi.

**Mfumo Muhimu:**
1. Angalia kama ujumbe ni `AgentExecutorResponse`
2. Changanua matokeo yaliyopangwa (mfano wa Pydantic)
3. Rudisha `True` au `False` kudhibiti usambazaji

Mtiririko wa kazi utatathmini masharti haya kwenye **viunganishi** ili kuamua ni mtekelezaji gani wa kuanzisha hatua inayofuata.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Hatua ya 4: Unda Mtekelezaji wa Maonyesho ya Kibinafsi

Watekelezaji ni vipengele vya mtiririko wa kazi vinavyofanya mabadiliko au athari za kando. Tunatumia mapambo ya `@executor` kuunda mtekelezaji wa kibinafsi unaoonyesha matokeo ya mwisho.

**Mambo Muhimu:**
- `@executor(id="...")` - Husajili kazi kama mtekelezaji wa mtiririko wa kazi
- `WorkflowContext[Never, str]` - Maelezo ya aina kwa pembejeo/mazao
- `ctx.yield_output(...)` - Hutoa matokeo ya mwisho ya mtiririko wa kazi


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Hatua ya 5: Pakia Vigezo vya Mazingira

Sanidi mteja wa LLM. Mfano huu unafanya kazi na:
- **GitHub Models** (Kiwango cha bure kwa kutumia tokeni ya GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Hatua ya 6: Unda Mawakala wa AI wenye Matokeo Yaliyopangiliwa

Tunaunda **mawakala watatu maalum**, kila mmoja akiwa amefungwa ndani ya `AgentExecutor`:

1. **availability_agent** - Hukagua upatikanaji wa hoteli kwa kutumia zana
2. **alternative_agent** - Hupendekeza miji mbadala (wakati hakuna vyumba)
3. **booking_agent** - Huhamasisha uhifadhi (wakati vyumba vinapatikana)

**Vipengele Muhimu:**
- `tools=[hotel_booking]` - Hutoa zana kwa wakala
- `response_format=PydanticModel` - Hulazimisha matokeo ya JSON yaliyopangiliwa
- `AgentExecutor(..., id="...")` - Hufunga wakala kwa matumizi ya mtiririko wa kazi


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Hatua ya 7: Jenga Mtiririko wa Kazi na Viunganishi vya Masharti

Sasa tunatumia `WorkflowBuilder` kujenga grafu yenye njia za masharti:

**Muundo wa Mtiririko wa Kazi:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Njia Muhimu:**
- `.set_start_executor(...)` - Huanzisha sehemu ya kuingilia
- `.add_edge(from, to, condition=...)` - Huongeza kiunganishi cha masharti
- `.build()` - Hukamilisha mtiririko wa kazi


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Hatua ya 8: Endesha Kesi ya Mtihani 1 - Jiji BILA Upatikanaji (Paris)

Hebu tujaribu njia ya **bila upatikanaji** kwa kuomba hoteli huko Paris (ambayo haina vyumba katika simulizi yetu).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Hatua ya 9: Endesha Kesi ya Mtihani 2 - Jiji LENYE Upatikanaji (Stockholm)

Sasa hebu tujaribu njia ya **upatikanaji** kwa kuomba hoteli huko Stockholm (ambayo ina vyumba katika simulizi yetu).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Mambo Muhimu na Hatua Zifuatazo

### ✅ Ulichojifunza:

1. **Mfumo wa WorkflowBuilder**
   - Tumia `.set_start_executor()` kufafanua sehemu ya kuanzia
   - Tumia `.add_edge(from, to, condition=...)` kwa njia za masharti
   - Piga `.build()` kukamilisha mtiririko wa kazi

2. **Njia za Masharti**
   - Kazi za masharti zinachunguza `AgentExecutorResponse`
   - Changanua matokeo yaliyopangwa ili kufanya maamuzi ya njia
   - Rudisha `True` kuamsha njia, `False` kuiruka

3. **Ujumuishaji wa Zana**
   - Tumia `@ai_function` kubadilisha kazi za Python kuwa zana za AI
   - Mawakala hutumia zana moja kwa moja inapohitajika
   - Zana zinarudisha JSON ambayo mawakala wanaweza kuchanganua

4. **Matokeo Yaliyopangwa**
   - Tumia mifano ya Pydantic kwa uchimbaji wa data salama kwa aina
   - Weka `response_format=MyModel` unapounda mawakala
   - Changanua majibu kwa `Model.model_validate_json()`

5. **Watekelezaji Maalum**
   - Tumia `@executor(id="...")` kuunda vipengele vya mtiririko wa kazi
   - Watekelezaji wanaweza kubadilisha data au kufanya athari za upande
   - Tumia `ctx.yield_output()` kutoa matokeo ya mtiririko wa kazi

### 🚀 Matumizi ya Kwenye Ulimwengu Halisi:

- **Uhifadhi wa Safari**: Angalia upatikanaji, pendekeza mbadala, linganisha chaguo
- **Huduma kwa Wateja**: Elekeza kulingana na aina ya tatizo, hisia, kipaumbele
- **E-commerce**: Angalia hesabu, pendekeza mbadala, shughulikia maagizo
- **Udhibiti wa Maudhui**: Elekeza kulingana na alama za sumu, maoni ya watumiaji
- **Mtiririko wa Idhini**: Elekeza kulingana na kiasi, jukumu la mtumiaji, kiwango cha hatari
- **Usindikaji wa Hatua Nyingi**: Elekeza kulingana na ubora wa data, ukamilifu

### 📚 Hatua Zifuatazo:

- Ongeza masharti magumu zaidi (vigezo vingi)
- Tekeleza mizunguko kwa usimamizi wa hali ya mtiririko wa kazi
- Ongeza mtiririko mdogo kwa vipengele vinavyoweza kutumika tena
- Jumuisha na API halisi (uhifadhi wa hoteli, mifumo ya hesabu)
- Ongeza usimamizi wa makosa na njia za dharura
- Onyesha mtiririko wa kazi kwa zana za kuona zilizojengwa ndani



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
